In [2]:
import pandas as pd 
import numpy as np
import matplotlib as plt
import pandas as pd
import nltk
import re
import numpy as np
from nltk import word_tokenize
from nltk.corpus import stopwords
from string import punctuation
import matplotlib.pyplot as plt
from collections import Counter
import pickle

#nltk.download()
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to C:\Users\Abhishek's
[nltk_data]     PC\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Abhishek's
[nltk_data]     PC\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
from keras.layers import Dense, Dropout, Flatten  
from keras.layers import Conv2D, MaxPooling2D
import keras
from keras.layers import Dense, Conv2D
from keras.layers import Flatten
from keras.layers import MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Activation
from keras.layers import BatchNormalization
from keras.layers import Dropout
from keras.models import Sequential
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn import model_selection
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
import re
from bs4 import BeautifulSoup
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import classification_report

Using TensorFlow backend.


##### One of the major problem with the data is .... its not uniform in nature, as in the body of the posts consists of images, videos, gifs, etc, the net analysis can only be completed if we take into account all sorts of data, besides that abstarcting the information might not prove to be healthy for model as the comments consists of sarcasm, biasness and other unethical information .... 

In [4]:
data=pd.read_csv('reddit-india-data.csv')
data = data.fillna('No Value')
feature_combine = data["title"] + data["comments"]
data = data.assign(feature_combine = feature_combine)

,flair,title,score,id,url,comms_num,body,author,comments,timestamp
0,AMA,I'm a Malayalee ABCD. Ask Me Anything.,1,cz98so,https://www.reddit.com/r/india/comments/cz98so...,42,I'm a Malayalee ABCD (American-Born Confused D...,simsim1000,Did you ever faced racism? \n\nAnd how is gir...,2019-09-04 08:34:35
1,AMA,What are some of the memorable posts of r/India??,85,cfj0kx,https://www.reddit.com/r/india/comments/cfj0kx...,67,As title suggests. Some posts are legends in R...,xxyyccb,Can't find it but some guy's mother posted he...,2019-07-20 20:37:58
2,AMA,Requesting AMA by Residents of Jammu & Kashmir.,4,crofzs,https://www.reddit.com/r/india/comments/crofzs...,18,"If there is any Residents of Jammu & Kashmir, ...",namanjha29,"On here, the accounts claiming to be Kashmir ...",2019-08-18 06:06:03
3,AMA,"Friends, Indians, countrymen, I cleared UPSC C...",1859,ba1o59,https://www.reddit.com/r/india/comments/ba1o59...,404,I am also super happy as I got the rank which ...,pseudoliberandu,Congrats. My neighbour aunty wants to know ab...,2019-04-06 20:14:18
4,AMA,So this happened yesterday [R],609,bcz7em,https://imgur.com/DJXElBz,77,NO VALUE,High24x7,the fact that they are acknowledging the prob...,2019-04-14 17:57:08
5,AMA,Congress social media head Divya Spandana dele...,78,bw5y0q,https://www.thehindu.com/news/national/congres...,16,NO VALUE,harddisc,IIRC she was on reddit and didnt answer much....,2019-06-03 16:44:45
6,AMA,How magician like Karan Singh do such magics? ...,6,c92763,https://www.reddit.com/r/india/comments/c92763...,11,NO VALUE,MrGogi61,Sleight of hand and practice. Would you like ...,2019-07-05 02:17:17
7,AMA,[Question] Has anyone here tried Paytm First?,5,cja9s9,https://www.reddit.com/r/india/comments/cja9s9...,1,I am thinking of buying Paytm First- does anyo...,shantanusri,Someone here did an AMA recently.,2019-07-30 01:28:04
8,AMA,AMA Request: A Kashmiri Pandit over 30 years old.,71,cnhzlf,https://www.reddit.com/r/india/comments/cnhzlf...,74,"Now that 370 and 35A have been abolished, a fe...",contraryview,I know one but he's 20. I'll try to contact h...,2019-08-08 20:23:43
9,AMA,I was searching for something and I stumbled u...,332,cd3xmx,https://i.imgur.com/FefhpHc.jpg,108,NO VALUE,SPYDER94,"Such a secular paper. Sorry, 10/10 or no AMA ...",2019-07-15 04:29:40


# Text Processing

### I have used 2 different approaches for the same 

## Method 1:

1. Extract the titles for each reddit post and tokenize them after removing stopwords and special characters and title for each post is stored in the form of the list, we then make a compile these lists to form a single lists for all the titles.

2. Loading the pretrained word embedding matrices from the state of the art glove model 

3. All the data points so that they are of the same size ,I do this by adding "ignore_nan" to sequences pertaining to a title untill they reach the desired length

4. Each word in the list of lists with its respective embedding matrix and replace "ignore_nan" with a 0 matrix 

5. Average all the non zero embedding matrixs pertaining to specific list and then store it and repeat this for all lists 

These now act as my inputs

In [5]:
def text_extractor(text,text_type):
    title_list=[]
    for i in range(len(text)):
        title_list.append(text[text_type][i])
    return title_list

def preProcessData(dataa):
    stopwords_en = list(set(stopwords.words('english')))
    def split(word): 
        return [char for char in word]   
    punchList = split(punctuation)

    print(stopwords_en)
    print('Punctuation :', punchList)

    wordTokenList = [word_tokenize(sent) for sent in dataa]
    lowercasingList = [[word.lower() for word in sentence] for sentence in wordTokenList]
    noStopWordList = [[word for word in sentence if word not in stopwords_en] for sentence in lowercasingList]
    noPunchList = [[re.sub(r'([^\s\w]|_)+', '', word) for word in sentence] for sentence in noStopWordList]
    PP_data = [[word for word in sentence if word] for sentence in noPunchList]
    return PP_data

def cleaner(PP_data):
    PP_corr=[]
    for title in PP_data:
        temp=[]
        for word in title:
            if word in embeddings_index.keys():
                    temp.append(word)
        PP_corr.append(temp)
    return PP_corr

def pad_features(titles,max_len):
    features=[]
    for title in titles:
        temp=[]
        for i in range(max_len):
            if i < len(title):
                temp.append(title[i])
            else:
                temp.append("ignore_nan")
        features.append(temp)
    return features


def vectorizer(summ_data,embeddings_index):
    summ_vec=[]
    for list_w in summ_data:
        temp=[]
        for word in list_w:
            if word in embeddings_index.keys():
                temp.append(embeddings_index[word])
        summ_vec.append(temp)
    return summ_vec

def sent_vectorizer(vector):
    sent_vect=[]
    for sent in vector:
        n=0
        temp=np.zeros(200)
        for word in sent:
            temp=temp+word
            n=n+1
        sent_vect.append(temp/n)
    return sent_vect

import os
glove_dir = 'glove'
embeddings_index = {} # empty dictionary
f = open(os.path.join(glove_dir, 'glove.6B.200d.txt'), encoding="utf-8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
z=np.zeros(200)
embeddings_index['ignore_nan']=z

def reshaper(t_x):
    x = np.reshape(t_x, (t_x.shape[0], 1,t_x.shape[1]))
    return x

def data_to_ready(data,embed):
    t_list=text_extractor(data,'title')
    pp_data=preProcessData(t_list)
    clean=cleaner(pp_data)
#     max_len=max_finder(clean)
    clean=pad_features(clean,256)
    final=vectorizer(clean,embed)
    final=sent_vectorizer(final)
    final=np.array(final)
#     re_final=reshaper(final)
#     return re_final
    return final

In [6]:
def shuffler(data):
    df_sh=shuffle(data)
    df_sh=df_sh.reset_index()
    return df_sh
def one_shotter(z,colt):
    y=z[colt]
    y=np.array(y)
    y=pd.get_dummies(y)
    return y

In [8]:
sh=shuffler(data)
y=one_shotter(sh,'flair')
y=np.array(y)
x=data_to_ready(sh,embeddings_index)


['what', 'they', 'been', 'y', 'myself', 'himself', 'those', 'under', "doesn't", 'mightn', 'there', 'any', "weren't", 'again', "couldn't", 'up', 'it', 'own', 'too', 'i', "you've", 'has', "you're", 'should', 's', 'is', 'by', "hadn't", 'which', 'she', 'few', 'o', 'at', 'whom', 'more', 'shouldn', 'did', "should've", 'her', 'these', 'have', 'into', 'his', 'do', 'other', 'ourselves', 'in', 'ma', 'further', 'wasn', 'was', "it's", 'an', "shouldn't", 'am', "needn't", "you'll", 'ours', 'just', 'aren', 'haven', 'yourself', 'themselves', 'didn', 'hadn', 'don', 'out', 'after', 'me', 'all', 'how', 'this', 'than', 'not', 'once', 'why', 'hasn', 'the', 'weren', 'as', 'above', 'between', 'yours', 'our', 'with', 'couldn', 'hers', "didn't", 'about', 'during', "aren't", 'of', "she's", 'most', 'them', 'won', 'him', 'where', 'being', 'a', 'off', "hasn't", "haven't", "won't", 'when', 'having', 'against', 'had', 'because', 'herself', "mustn't", "that'll", 'mustn', 'my', 're', "mightn't", 'some', 'be', 'to', 'd

In [13]:
from keras.layers import Dense, Dropout, Flatten  
from keras.layers import Conv2D, MaxPooling2D
import keras
from keras.layers import Dense, Conv2D
from keras.layers import Flatten
from keras.layers import MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Activation
from keras.layers import BatchNormalization
from keras.layers import Dropout
from keras.models import Sequential
from keras import backend as K
from keras.preprocessing.image import load_img


from keras import optimizers

## Methos 1A:
#### training a deep neural net on the processed input and oneshotted output

In [14]:
model = Sequential()    
model.add(Dense(256, activation='relu')) 
model.add(Dense(128, activation='sigmoid'))
model.add(Dense(64, activation='relu'))
model.add(Dense(11, activation='softmax')) 

W0908 21:52:29.607711 12108 deprecation_wrapper.py:119] From d:\anacoda\envs\machine\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.



In [27]:
model.compile(loss=keras.losses.categorical_crossentropy,  
       optimizer=keras.optimizers.Adadelta(),       
       metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 256)               51456     
_________________________________________________________________
dense_2 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_3 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_4 (Dense)              (None, 11)                715       
Total params: 93,323
Trainable params: 93,323
Non-trainable params: 0
_________________________________________________________________


In [26]:
model.fit(x[0:700],y[0:700],epochs=100,validation_data=(x[700:800],y[700:800] ))

W0908 21:55:20.766204 12108 deprecation_wrapper.py:119] From d:\anacoda\envs\machine\lib\site-packages\keras\backend\tensorflow_backend.py:3295: The name tf.log is deprecated. Please use tf.math.log instead.

W0908 21:55:20.994102 12108 deprecation.py:323] From d:\anacoda\envs\machine\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0908 21:55:21.064914 12108 deprecation_wrapper.py:119] From d:\anacoda\envs\machine\lib\site-packages\keras\backend\tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Train on 700 samples, validate on 100 samples
Epoch 1/100
700/700 [==============================] - 1s 1ms/step - loss: 2.4035 - acc: 0.0943 - val_loss: 2.3912 - val_acc: 0.0600
Epoch 2/100
700/700 [==============================] - 0s 51us/step - loss: 2.3692 - acc: 0.0986 - val_loss: 2.3873 - val_acc: 0.1400
Epoch 3/100
700/700 [==============================] - 0s 50us/step - loss: 2.3720 - acc: 0.0743 - val_loss: 2.4070 - val_acc: 0.1400
Epoch 4/100
700/700 [==============================] - 0s 53us/step - loss: 2.3614 - acc: 0.1043 - val_loss: 2.4009 - val_acc: 0.0900
Epoch 5/100
700/700 [==============================] - 0s 53us/step - loss: 2.3574 - acc: 0.1086 - val_loss: 2.3934 - val_acc: 0.1000
Epoch 6/100
700/700 [==============================] - 0s 53us/step - loss: 2.3626 - acc: 0.0786 - val_loss: 2.3824 - val_acc: 0.0900
Epoch 7/100
700/700 [==============================] - 0s 53us/step - loss: 2.3595 - acc: 0.0971 - val_loss: 2.4011 - val_acc: 0.0900
Epoch 8/100
700/7

In [29]:
from keras import optimizers
from keras.layers import SimpleRNN,Embedding,LSTM,Dropout
from keras.layers import Dense, Dropout, Flatten  
from keras.layers import Conv2D, MaxPooling2D
model = Sequential()
model.add(LSTM(512,input_shape=(1,200),return_sequences=False))#True = many to many
model.add(Dropout(0.3))
model.add(Dense(256,kernel_initializer='normal',activation='relu'))
# model.add(Dense(64,kernel_initializer='normal',activation='linear'))
model.add(Dense(32,kernel_initializer='normal',activation='sigmoid'))
model.add(Dense(11,kernel_initializer='normal',activation='relu'))
model.compile(loss='mse',optimizer ='adam',metrics=['accuracy'])
# scores = model.evaluate(train_x[0:5],train_y[0:5],verbose=1,batch_size=1)

W0908 21:57:08.623966 12108 deprecation.py:506] From d:\anacoda\envs\machine\lib\site-packages\keras\backend\tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


## Method 1B:
#### Reshaping our input points and training a LSTM network followed a DNN on our reshaped input and oneshotted out[ut

In [32]:
p_x=reshaper(x)

In [33]:
model.fit(p_x[0:700],y[0:700],epochs=100,validation_data=(p_x[700:800],y[700:800] ))

Train on 700 samples, validate on 100 samples
Epoch 1/100
700/700 [==============================] - 2s 2ms/step - loss: 0.0884 - acc: 0.1071 - val_loss: 0.0859 - val_acc: 0.0900
Epoch 2/100
700/700 [==============================] - 1s 903us/step - loss: 0.0858 - acc: 0.1100 - val_loss: 0.0856 - val_acc: 0.0600
Epoch 3/100
700/700 [==============================] - 1s 858us/step - loss: 0.0857 - acc: 0.0871 - val_loss: 0.0856 - val_acc: 0.0600
Epoch 4/100
700/700 [==============================] - 1s 856us/step - loss: 0.0858 - acc: 0.0786 - val_loss: 0.0856 - val_acc: 0.0900
Epoch 5/100
700/700 [==============================] - 1s 876us/step - loss: 0.0857 - acc: 0.0857 - val_loss: 0.0852 - val_acc: 0.1400
Epoch 6/100
700/700 [==============================] - 1s 848us/step - loss: 0.0857 - acc: 0.1071 - val_loss: 0.0852 - val_acc: 0.1300
Epoch 7/100
700/700 [==============================] - 1s 892us/step - loss: 0.0854 - acc: 0.1200 - val_loss: 0.0852 - val_acc: 0.1100
Epoch 8/100

Epoch 61/100
700/700 [==============================] - 1s 875us/step - loss: 0.0519 - acc: 0.5343 - val_loss: 0.0591 - val_acc: 0.4800
Epoch 62/100
700/700 [==============================] - 1s 852us/step - loss: 0.0520 - acc: 0.5300 - val_loss: 0.0595 - val_acc: 0.4800
Epoch 63/100
700/700 [==============================] - 1s 858us/step - loss: 0.0516 - acc: 0.5300 - val_loss: 0.0595 - val_acc: 0.4700
Epoch 64/100
700/700 [==============================] - 1s 849us/step - loss: 0.0518 - acc: 0.5343 - val_loss: 0.0598 - val_acc: 0.4700
Epoch 65/100
700/700 [==============================] - 1s 853us/step - loss: 0.0517 - acc: 0.5329 - val_loss: 0.0601 - val_acc: 0.4600
Epoch 66/100
700/700 [==============================] - 1s 875us/step - loss: 0.0511 - acc: 0.5357 - val_loss: 0.0604 - val_acc: 0.4800
Epoch 67/100
700/700 [==============================] - 1s 839us/step - loss: 0.0510 - acc: 0.5329 - val_loss: 0.0598 - val_acc: 0.4800
Epoch 68/100
700/700 [==========================

# Model 2:

Here I use basic models on the same 
 
* instead of assigning glove embedding matix to each word , here I just remove the stopwords and special characters from titles of each post and then join these titles back to form a list of processed title strings

this now acts as my input

In [7]:
from sklearn.utils import shuffle

def joiner(data):
    input_corrected = [" ".join(i) for i in data]
    return input_corrected
def data_prep(data,text_type):
    sh=shuffler(data)
    y=sh['flair']
    x=joiner(preProcessData(text_extractor(sh,text_type)))
    return x,y

Now I use 3 approaches on the same input  -
1. Random Forest
2. Lineer SVM
3. Logistical Regression 

as these 3 are the most common methods used for classification

instead of oneshotting the flairs (ouput) , i use them directly

In [44]:
def randomfr(x_train, x_test, y_train, y_test):
    model = Pipeline([('vect', CountVectorizer()),
                  ('tfidf', TfidfTransformer()),
                  ('clf', RandomForestClassifier(n_estimators = 900)),
                 ])
    model.fit(x_train, y_train)

    y_pred = model.predict(x_test)
    filename = 'rfr_model.sav'
    pickle.dump(model, open(filename, 'wb'))
    print('accuracy %s' % accuracy_score(y_pred, y_test))
    print(classification_report(y_test, y_pred,target_names=flairs))


In [40]:
def linearsvm(x_train, x_test, y_train, y_test):  
    model= Pipeline([('vect', CountVectorizer()),
                  ('tfidf', TfidfTransformer()),
                  ('clf', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-2, max_iter=10, tol=None)),
                 ])
    model.fit(x_train, y_train)

    y_pred = model.predict(x_test)
    filename = 'lsvm_model.sav'
    pickle.dump(model, open(filename, 'wb'))
    print('accuracy %s' % accuracy_score(y_pred, y_test))
    print(classification_report(y_test, y_pred,target_names=flairs))

In [41]:
def logisticregg(x_train, x_test, y_train, y_test):
    model = Pipeline([('vect', CountVectorizer()),
                  ('tfidf', TfidfTransformer()),
                  ('clf', LogisticRegression(n_jobs=1, C=1e5)),
                 ])
    model.fit(x_train, y_train)

    y_pred = model.predict(x_test)
    filename = 'lg_model.sav'
    pickle.dump(model, open(filename, 'wb'))
    print('accuracy %s' % accuracy_score(y_pred, y_test))
    print(classification_report(y_test, y_pred,target_names=flairs))

In [30]:
x,y=data_prep(data,'title')
x_train=x[0:750]
y_train=y[0:750]
x_test=x[750:]
y_test[750:]
flairs=y.unique()

['what', 'they', 'been', 'y', 'myself', 'himself', 'those', 'under', "doesn't", 'mightn', 'there', 'any', "weren't", 'again', "couldn't", 'up', 'it', 'own', 'too', 'i', "you've", 'has', "you're", 'should', 's', 'is', 'by', "hadn't", 'which', 'she', 'few', 'o', 'at', 'whom', 'more', 'shouldn', 'did', "should've", 'her', 'these', 'have', 'into', 'his', 'do', 'other', 'ourselves', 'in', 'ma', 'further', 'wasn', 'was', "it's", 'an', "shouldn't", 'am', "needn't", "you'll", 'ours', 'just', 'aren', 'haven', 'yourself', 'themselves', 'didn', 'hadn', 'don', 'out', 'after', 'me', 'all', 'how', 'this', 'than', 'not', 'once', 'why', 'hasn', 'the', 'weren', 'as', 'above', 'between', 'yours', 'our', 'with', 'couldn', 'hers', "didn't", 'about', 'during', "aren't", 'of', "she's", 'most', 'them', 'won', 'him', 'where', 'being', 'a', 'off', "hasn't", "haven't", "won't", 'when', 'having', 'against', 'had', 'because', 'herself', "mustn't", "that'll", 'mustn', 'my', 're', "mightn't", 'some', 'be', 'to', 'd

In [12]:
randomfr(x_train,x_test,y_train, y_test)

accuracy 0.5410447761194029
                    precision    recall  f1-score   support

          Politics       0.75      0.46      0.57        26
            Sports       0.47      0.69      0.56        26
  Business/Finance       0.16      0.43      0.23        23
          AskIndia       0.65      0.52      0.58        25
              Food       1.00      0.92      0.96        24
     Non-Political       0.89      0.62      0.73        26
       Photography       0.53      0.61      0.56        33
               AMA       0.53      0.38      0.44        26
    Policy/Economy       0.77      0.40      0.53        25
Science/Technology       0.70      0.47      0.56        30
     [R]eddiquette       0.00      0.00      0.00         4

          accuracy                           0.54       268
         macro avg       0.59      0.50      0.52       268
      weighted avg       0.63      0.54      0.56       268



d:\anacoda\envs\machine\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [13]:
linearsvm(x_train,x_test,y_train, y_test)

accuracy 0.6231343283582089
                    precision    recall  f1-score   support

          Politics       0.82      0.54      0.65        26
            Sports       0.50      0.65      0.57        26
  Business/Finance       0.33      0.22      0.26        23
          AskIndia       0.61      0.68      0.64        25
              Food       0.86      1.00      0.92        24
     Non-Political       0.77      0.77      0.77        26
       Photography       0.54      0.64      0.58        33
               AMA       0.48      0.50      0.49        26
    Policy/Economy       0.62      0.60      0.61        25
Science/Technology       0.69      0.67      0.68        30
     [R]eddiquette       1.00      0.25      0.40         4

          accuracy                           0.62       268
         macro avg       0.66      0.59      0.60       268
      weighted avg       0.63      0.62      0.62       268



In [14]:
logisticregg(x_train,x_test,y_train, y_test)

d:\anacoda\envs\machine\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
d:\anacoda\envs\machine\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


accuracy 0.6156716417910447
                    precision    recall  f1-score   support

          Politics       0.78      0.54      0.64        26
            Sports       0.65      0.65      0.65        26
  Business/Finance       0.27      0.35      0.30        23
          AskIndia       0.67      0.64      0.65        25
              Food       0.92      0.92      0.92        24
     Non-Political       0.76      0.73      0.75        26
       Photography       0.58      0.64      0.61        33
               AMA       0.48      0.50      0.49        26
    Policy/Economy       0.54      0.56      0.55        25
Science/Technology       0.65      0.67      0.66        30
     [R]eddiquette       1.00      0.25      0.40         4

          accuracy                           0.62       268
         macro avg       0.66      0.59      0.60       268
      weighted avg       0.64      0.62      0.62       268



Now instead of only using title , I use title and the comments as the newer methos were giving better results but they werent satisfactory

In [35]:
x,y=data_prep(sh,'feature_combine')
x_train=x[0:750]
y_train=y[0:750]
x_test=x[750:]
y_test[750:]

['what', 'they', 'been', 'y', 'myself', 'himself', 'those', 'under', "doesn't", 'mightn', 'there', 'any', "weren't", 'again', "couldn't", 'up', 'it', 'own', 'too', 'i', "you've", 'has', "you're", 'should', 's', 'is', 'by', "hadn't", 'which', 'she', 'few', 'o', 'at', 'whom', 'more', 'shouldn', 'did', "should've", 'her', 'these', 'have', 'into', 'his', 'do', 'other', 'ourselves', 'in', 'ma', 'further', 'wasn', 'was', "it's", 'an', "shouldn't", 'am', "needn't", "you'll", 'ours', 'just', 'aren', 'haven', 'yourself', 'themselves', 'didn', 'hadn', 'don', 'out', 'after', 'me', 'all', 'how', 'this', 'than', 'not', 'once', 'why', 'hasn', 'the', 'weren', 'as', 'above', 'between', 'yours', 'our', 'with', 'couldn', 'hers', "didn't", 'about', 'during', "aren't", 'of', "she's", 'most', 'them', 'won', 'him', 'where', 'being', 'a', 'off', "hasn't", "haven't", "won't", 'when', 'having', 'against', 'had', 'because', 'herself', "mustn't", "that'll", 'mustn', 'my', 're', "mightn't", 'some', 'be', 'to', 'd

In [45]:
randomfr(x_train,x_test,y_train, y_test)

accuracy 0.8022388059701493
                    precision    recall  f1-score   support

          AskIndia       0.75      0.91      0.82        23
     Non-Political       0.56      0.82      0.67        22
Science/Technology       0.86      0.55      0.67        22
            Sports       0.86      0.73      0.79        26
       Photography       0.94      0.94      0.94        32
    Policy/Economy       0.83      0.87      0.85        23
          Politics       0.68      0.82      0.74        28
  Business/Finance       0.77      0.85      0.81        27
              Food       0.92      0.67      0.77        33
               AMA       0.96      0.93      0.95        28
     [R]eddiquette       1.00      0.25      0.40         4

          accuracy                           0.80       268
         macro avg       0.83      0.76      0.76       268
      weighted avg       0.82      0.80      0.80       268



In [42]:
linearsvm(x_train,x_test,y_train, y_test)

accuracy 0.7798507462686567
                    precision    recall  f1-score   support

          AskIndia       0.77      0.87      0.82        23
     Non-Political       0.71      0.55      0.62        22
Science/Technology       0.87      0.59      0.70        22
            Sports       0.76      0.73      0.75        26
       Photography       0.83      0.62      0.71        32
    Policy/Economy       0.79      0.96      0.86        23
          Politics       0.70      0.93      0.80        28
  Business/Finance       0.77      0.89      0.83        27
              Food       0.79      0.82      0.81        33
               AMA       0.83      0.89      0.86        28
     [R]eddiquette       1.00      0.25      0.40         4

          accuracy                           0.78       268
         macro avg       0.80      0.74      0.74       268
      weighted avg       0.79      0.78      0.77       268



In [43]:
logisticregg(x_train,x_test,y_train, y_test)

d:\anacoda\envs\machine\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
d:\anacoda\envs\machine\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


accuracy 0.7910447761194029
                    precision    recall  f1-score   support

          AskIndia       0.75      0.91      0.82        23
     Non-Political       0.55      0.82      0.65        22
Science/Technology       0.83      0.68      0.75        22
            Sports       0.85      0.65      0.74        26
       Photography       0.92      0.69      0.79        32
    Policy/Economy       0.88      0.91      0.89        23
          Politics       0.75      0.86      0.80        28
  Business/Finance       0.70      0.85      0.77        27
              Food       0.90      0.79      0.84        33
               AMA       0.92      0.86      0.89        28
     [R]eddiquette       1.00      0.25      0.40         4

          accuracy                           0.79       268
         macro avg       0.82      0.75      0.76       268
      weighted avg       0.81      0.79      0.79       268

